## Design Pattern


### SOLID 원칙

객체지향 설계 5대 원칙 (SRP, OCP, LSP, ISP, DIP)을 의미한다.


#### 1. Single Responsibility Principle

단일 책임 원칙, 한 클래스는 단 하나의 책임만 가진다. 만약 여러 책임이 합쳐지면, 변경이나 유지보수가 어려워진다.

#### 2. OCP (Open/Closed Principle)

개방-폐쇄 원칙, 확장에는 열려 있고, 변경에는 닫혀 있어야 한다. 기존 코드를 수정하지 않고, 새로운 기능이나 구현을 추가해도 확장이 가능하도록 설계한다.

#### 3. LSP (Liskov Substitution Principle)

리스코프 치환 원칙, 자식 클래스는 부모 클래스로 대체(치환)될 수 있어야 한다. 즉, 부모 클래스(인터페이스)가 기대하는 동작이나 규약을 자식 클래스도 위배하지 않아야 한다.

#### 4. ISP (Interface Segregation Principle)

인터페이스 분리 원칙, 덩치 큰 인터페이스 하나를 만들지 말고, 작고 구체적인 인터페이스 여러 개로 분리한다. 불필요한 메서드를 구현할 의무가 생기지 않도록 설계한다.

#### 5. DIP (Dependency Inversion Principle)

의존성 역전 원칙, 상위(고수준) 모듈이 하위(저수준) 모듈 세부 구현에 의존하지 않도록 하고, 추상화(인터페이스)에 의존하게 만든다.

### Design Pattern

OOP에서 자주 사용되는 해결책 패턴을 의미한다. 자주 발생하는 설계 문제를 효율적으로 해결하고, 가독성과 유지보수 및 원활한 협업을 목표로 한다.

#### 1. Singleton

- 애플리케이션에서 특정 클래스의 인스턴스가 단 하나만 존재하도록 보장한다.
- 인스턴스가 여러 개 생기면 안 되거나, 전역적으로 공유되는 리소스(예: 설정값, 로거) 등에 주로 사용한다.

#### 2. Factory Method

- 객체 생성 로직을 서브클래스에 위임하는 방식이다.
- 상위 클래스(또는 인터페이스)는 객체를 생성하는 **‘팩토리 메서드’** 를 갖고, 실제 구현은 자식 클래스가 결정한다.
- 새로운 구체 클래스가 추가되어도 상위 클래스 코드를 수정하지 않고 확장할 수 있다.

#### 3. Strategy

- 알고리즘(또는 행위)을 캡슐화하여 런타임에 교체 가능하도록 만든다. 예를 들어, 로직을 여러 가지 전략으로 구분하고, 상황에 따라 다른 전략 객체를 주입해 사용한다.

#### 4. Observer

- **주체(Subject)** 와 이를 관찰하는 옵저버(Observer) 간 일 대 다 의존 관계를 설정한다.
- 주체의 상태가 변하면, 연결된 옵저버들에게 자동으로 알림을 보낸다.
- 이벤트 기반 시스템, GUI, 금융 시세 업데이트 등에서 흔히 사용한다.

#### 5. Decorator

- 기존 객체에 새로운 기능을 동적으로 추가하는 방법이다.
- 상속 대신 **랩핑(Wrapping)** 을 통해 객체에 책임을 덧붙일 수 있다.